<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Sparse_BIC_9_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (Sparse)

In [28]:
# Function to generate CPDs for the sparse structure
def generate_cpds_sparse():
    # Generate random probabilities for each node independently
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()

    irt_given_ir_probs = np.random.rand(3, 3)
    irt_given_ir_probs /= irt_given_ir_probs.sum(axis=0, keepdims=True)

    ei_given_ir_irt_probs = np.random.rand(3, 3, 3)
    ei_given_ir_irt_probs /= ei_given_ir_irt_probs.sum(axis=0, keepdims=True)

    geo_probs = np.random.rand(3)
    geo_probs /= geo_probs.sum()

    ue_given_geo_probs = np.random.rand(3, 3)
    ue_given_geo_probs /= ue_given_geo_probs.sum(axis=0, keepdims=True)

    ms_given_ei_geo_probs = np.random.rand(3, 3, 3)
    ms_given_ei_geo_probs /= ms_given_ei_geo_probs.sum(axis=0, keepdims=True)

    irp_given_ir_irt_probs = np.random.rand(3, 3, 3)
    irp_given_ir_irt_probs /= irp_given_ir_irt_probs.sum(axis=0, keepdims=True)

    inf_given_ms_irp_probs = np.random.rand(3, 3, 3)
    inf_given_ms_irp_probs /= inf_given_ms_irp_probs.sum(axis=0, keepdims=True)

    # SP depends on each of the other nodes (sparsely) without interactions between them
    sp_probs = np.random.rand(3, 3, 3, 3, 3, 3, 3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, irt_given_ir_probs, ei_given_ir_irt_probs, geo_probs, ue_given_geo_probs, ms_given_ei_geo_probs, irp_given_ir_irt_probs, inf_given_ms_irp_probs, sp_probs

# Function to generate and save samples with the sparse structure
def generate_and_save_samples_sparse(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample each node individually
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]

        irt_probs_given_ir = irt_probs[:, ir_state_idx]
        irt_state_idx = np.random.choice(3, p=irt_probs_given_ir)
        irt_state = ['decreasing', 'steady', 'increasing'][irt_state_idx]

        ei_probs_given_ir_irt = ei_probs[:, ir_state_idx, irt_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir_irt)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]

        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['recession', 'stable', 'growth'][geo_state_idx]

        ue_probs_given_geo = ue_probs[:, geo_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_geo)
        ue_state = ['high', 'medium', 'low'][ue_state_idx]

        ms_probs_given_ei_geo = ms_probs[:, ei_state_idx, geo_state_idx]
        ms_state_idx = np.random.choice(3, p=ms_probs_given_ei_geo)
        ms_state = ['negative', 'neutral', 'positive'][ms_state_idx]

        irp_probs_given_ir_irt = irp_probs[:, ir_state_idx, irt_state_idx]
        irp_state_idx = np.random.choice(3, p=irp_probs_given_ir_irt)
        irp_state = ['loose', 'neutral', 'tight'][irp_state_idx]

        inf_probs_given_ms_irp = inf_probs[:, ms_state_idx, irp_state_idx]
        inf_state_idx = np.random.choice(3, p=inf_probs_given_ms_irp)
        inf_state = ['deflation', 'stable', 'inflation'][inf_state_idx]

        # Calculate SP probability based on the state of each node (sparse dependency on each)
        sp_probs_given_all = sp_probs[:, ir_state_idx, irt_state_idx, ei_state_idx, geo_state_idx, ue_state_idx, ms_state_idx, irp_state_idx, inf_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_all)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]

        # Append sample data to output list including probabilities for all nodes
        output_data.append({
            'IR_State': ir_state,
            'IRT_State': irt_state,
            'EI_State': ei_state,
            'GEO_State': geo_state,
            'UE_State': ue_state,
            'MS_State': ms_state,
            'IRP_State': irp_state,
            'INF_State': inf_state,
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_all]),
            'Chosen_SP_State': sp_state
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

for size in sample_sizes:
    ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, sp_probs = generate_cpds_sparse()
    generate_and_save_samples_sparse(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, irp_probs, inf_probs, sp_probs, size, f'combined_probabilities_sparse_{size}.csv')

print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+-------------+------------+-------------+------------+------------+-------------+-------------+-------------------------------------------------+-------------------+
|    | IR_State   | IRT_State   | EI_State   | GEO_State   | UE_State   | MS_State   | IRP_State   | INF_State   | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |
+====+============+=============+============+=============+============+============+=============+=============+=================================================+===================+
|  0 | medium     | decreasing  | poor       | recession   | medium     | positive   | tight       | deflation   | 0.3772, 0.4019, 0.2208                          | stable            |
+----+------------+-------------+------------+-------------+------------+------------+-------------+-------------+-------------------------------------------------+-------------------+
|  1 | low        

# LBN Sparse BIC & Entropy

In [29]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dataset for the current sample size
    data_file = f'combined_probabilities_sparse_{sample_size}.csv'
    df = pd.read_csv(data_file)

    # Manually encode categorical variables for IR, IRT, EI, GEO, UE, MS, IRP, INF, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    irt_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    geo_map = {'recession': 0, 'stable': 1, 'growth': 2}
    ue_map = {'high': 0, 'medium': 1, 'low': 2}
    ms_map = {'negative': 0, 'neutral': 1, 'positive': 2}
    irp_map = {'loose': 0, 'neutral': 1, 'tight': 2}
    inf_map = {'deflation': 0, 'stable': 1, 'inflation': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR_State'].map(ir_map)
    df['IRT_encoded'] = df['IRT_State'].map(irt_map)
    df['EI_encoded'] = df['EI_State'].map(ei_map)
    df['GEO_encoded'] = df['GEO_State'].map(geo_map)
    df['UE_encoded'] = df['UE_State'].map(ue_map)
    df['MS_encoded'] = df['MS_State'].map(ms_map)
    df['IRP_encoded'] = df['IRP_State'].map(irp_map)
    df['INF_encoded'] = df['INF_State'].map(inf_map)
    df['SP_encoded'] = df['Chosen_SP_State'].map(sp_map)

    # Split the data into training and test sets
    X = df[['IR_encoded', 'IRT_encoded', 'EI_encoded', 'GEO_encoded', 'UE_encoded', 'MS_encoded', 'IRP_encoded', 'INF_encoded']]
    y = df['SP_encoded']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

    # Concatenate X_train and y_train to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)
    df_test = pd.concat([X_test, y_test], axis=1)

    # Define the sparse structure with priors (fixed edges only pointing to SP)
    priors = [
        ('IR_encoded', 'SP_encoded'), ('IRT_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded'),
        ('GEO_encoded', 'SP_encoded'), ('UE_encoded', 'SP_encoded'), ('MS_encoded', 'SP_encoded'),
        ('IRP_encoded', 'SP_encoded'), ('INF_encoded', 'SP_encoded')
    ]

    # Perform Hill Climb Search with priors on the training data
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=BicScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Try fitting parameters using BayesianEstimator on the training data
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # Predict on test data and calculate K-L divergence
    try:
        # Generate predictions for SP_encoded based on the learned model
        inference = best_model.predict(X_test)
        predicted_probabilities = inference['SP_encoded']

        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))

# Save the K-L divergence results
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_bic_sparse.csv', index=False)

print("\nAll results have been saved.")



Processing sample size: 50


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IR_encoded', 'INF_encoded'), ('INF_encoded', 'SP_encoded'), ('IRT_encoded', 'SP_encoded'), ('IRT_encoded', 'IRP_encoded'), ('IRP_encoded', 'SP_encoded'), ('IRP_encoded', 'IR_encoded'), ('IRP_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded'), ('GEO_encoded', 'SP_encoded'), ('UE_encoded', 'SP_encoded'), ('MS_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/15 [00:00<?, ?it/s]

K-L Divergence: 8.8703
Standard Deviation: 0.4320

Processing sample size: 100


...

K-L Divergence: 0.0241
Standard Deviation: 0.0757

Processing sample size: 20000


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded'), ('EI_encoded', 'GEO_encoded'), ('IRT_encoded', 'SP_encoded'), ('IRT_encoded', 'IR_encoded'), ('IRT_encoded', 'IRP_encoded'), ('IRT_encoded', 'EI_encoded'), ('IRP_encoded', 'SP_encoded'), ('IRP_encoded', 'IR_encoded'), ('IRP_encoded', 'MS_encoded'), ('IRP_encoded', 'INF_encoded'), ('GEO_encoded', 'SP_encoded'), ('GEO_encoded', 'UE_encoded'), ('UE_encoded', 'SP_encoded'), ('MS_encoded', 'SP_encoded'), ('MS_encoded', 'GEO_encoded'), ('INF_encoded', 'SP_encoded'), ('INF_encoded', 'MS_encoded')]
Model fitted successfully.


  0%|          | 0/2394 [00:00<?, ?it/s]

K-L Divergence: 0.0373
Standard Deviation: 0.0932

All results have been saved.
